<a href="https://colab.research.google.com/github/Tahira2910/ETG_Python_Projects/blob/main/Cryptocurrency_Price_Prediction_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I’ll start the task of Cryptocurrency price prediction by importing the necessary Python libraries and the dataset we need. For this task, I will collect the latest Bitcoin prices data from Yahoo Finance, using the yfinance API. This will help you collect the latest data each time you run this code:

In [ ]:
pip install autots

In [ ]:
import pandas as pd

import yfinance as yf    # collecting the latest Bitcoin prices data from Yahoo Finance, using the yfinance API


import datetime          # It allows you to create, manipulate, format, and perform calculations on dates and times


# The date class represents a date (year, month, day) without time, while the timedelta class represents a duration or difference between two dates or times
from datetime import date, timedelta 

# This line creates a variable named "today" and assigns it the value of 
# the current date. The date.today() function returns the current local date based on the system's clock. 
today = date.today()

In [ ]:
d1 = today.strftime("%Y-%m-%d")  # converts the current date (today) into a string format with the specified format "%Y-%m-%d"
end_date = d1                    # assigns the value of the formatted date string (d1) to the variable end_date
d2 = date.today() - timedelta(days=730)  #calculates a new date by subtracting 730 days (two years) from the current date (date.today())
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD',   #BTC-USD represents the stock symbol for Bitcoin,  'USD' indicates that the value of Bitcoin is being measured in U.S. dollars.
                      start=start_date,  #specifies the start date and end date for retrieving the financial data
                      end=end_date, 
                      progress=False)  #disable the progress bar display during the download.
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]] #selects specific columnsfrom the DataFrame data and assigns the resulting DataFrame back to data
data.reset_index(drop=True, inplace=True) #drop=True--> parameter is used to discard the previous index, inplace=True--> modifying the original DataFrame.
print(data.head())


In [ ]:
data.shape #shows number of rows and the number of columns in the DataFrame.

In [ ]:
import plotly.graph_objects as go # used this module to create a candlestick chart 
# creates new figure object named figure  using the go.Figure() constructor
# The data parameter is specified as a list containing a single element, which is the go.Candlestick() object
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],  # represents the opening prices of the candlestick chart
                                        high=data["High"], #  represents the high prices of the candlestick char
                                        low=data["Low"],    # represents the low prices of the candlestick chart
                                        close=data["Close"])])  #  represents the closing prices of the candlestick chart
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False) #  hides the range slider for the x-axis.
figure.show()

In [ ]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

In [ ]:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

The Close column in the dataset contains the values we need to predict. So, let’s have a look at the correlation of all the columns in the data concerning the Close column:

In [ ]:
correlation = data.corr() # calculates the correlation matrix between all columns in the DataFrame data using the corr() method. 

# prints the correlation values for the "Close" column. It sorts these 
# correlation values in descending order using the sort_values() method with the ascending=False parameter. 
print(correlation["Close"].sort_values(ascending=False)) 

In [ ]:
from autots import AutoTS # library used for automated time series forecasting.


# creates an instance of the AutoTS class and assigns it to the variable model
# we want to forecast 30 periods into the future.
# implies that the frequency will be automatically inferred from the data.
# use for combining multiple forecasting models. Here, the 'simple' ensemble method is chosen.
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')


# Specifies the column name in the DataFrame that contains the dates or timestamps.
# value_col=Close--> contains the values to be forecaste
# id_col=None--> indicating that there are no individual time series to consider.
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast # extracts the forecasted values
print(forecast)